# Homework: Serverless Deep Learning (week 9)

In this homework, we'll deploy the bees vs wasps model we trained in the 
[previous homework](../08-deep-learning/README.md).

Model can be downloaded from here (is available in local repo as `bees-wasps.h5`):

https://github.com/alexeygrigorev/large-datasets/releases/download/wasps-bees/bees-wasps.h5

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.lite as tflite

keras_model_path = 'bees-wasps.h5'
tflite_model_path = 'bees-wasps.tflite'
img_url = 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'

In [12]:
keras_model = keras.models.load_model(keras_model_path)

converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)

tflite_model = converter.convert()

with open(tflite_model_path, 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\larki\AppData\Local\Temp\tmpiwylyp68\assets


INFO:tensorflow:Assets written to: C:\Users\larki\AppData\Local\Temp\tmpiwylyp68\assets


## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

In [14]:
import os

tflite_model_size = os.stat(tflite_model_path).st_size
print(f"{tflite_model_path} : {round(tflite_model_size/(1024*1024),3)}MB")

bees-wasps.tflite : 42.788MB


## Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output. 

What's the output index for this model?

In [18]:
interpreter = tflite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

print("input_index: ", input_index, "\n"
      "output_index: ", output_index)

input_index:  0 
output_index:  13


## Question 3

Now we need to turn the image into numpy array and pre-process it (the way we did in prev homework).

After the pre-processing, what's the value in the first pixel, the R channel?

In [2]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url) -> Image:
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def preprocess_image(img: Image, target_size) -> np.ndarray:
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    x = np.asarray(img)/255
    return x.astype(np.float32)

img = download_image(img_url)
x = preprocess_image(img, (150, 150))

display(x.shape, x[0][0][0])

(150, 150, 3)

0.94509804

## Question 4

Now let's apply this model to this image. What's the output of the model?

In [34]:
interpreter.set_tensor(input_index, [x])
interpreter.invoke()
output_data = interpreter.get_tensor(output_index)

# class indicies from prev homework = {'bee': 0, 'wasp': 1}
output_data

array([[0.6592137]], dtype=float32)

In [2]:
from lambda_function import predict

predict(img_url)

{'result': '0.6592137', 'bee': 0, 'wasp': 1}